In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np


In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [8]:
n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [10]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [17]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [18]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [19]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [20]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [21]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [22]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [23]:
n_epochs = 40
batch_size = 50

In [26]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            #print(X_batch, y_batch)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9126
1 배치 데이터 정확도: 0.88 검증 세트 정확도: 0.9314
2 배치 데이터 정확도: 0.88 검증 세트 정확도: 0.9384
3 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9476
4 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9526
5 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9538
6 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9574
7 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9608
8 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9632
9 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.966
10 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9668
11 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9668
12 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9704
13 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9704
14 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.97
15 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9712
16 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9718
17 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9722
18 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9736
19 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9738
20 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9738
21 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9734
22 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9746
23 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9746
24 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9752
25 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9758
26 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9764

In [29]:
with tf.Session() as sess:
    saver.restore(sess, "./tf_logs/my_model_final.ckpt") # 또는 save_path를 사용합니다
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./tf_logs/my_model_final.ckpt


In [30]:
print(Z)

[[  0.06123117  -1.8012471    2.5454156    5.380114    -7.7588005
   -2.1042051  -13.064613    12.176681     0.11012305   1.618759  ]
 [  1.8441111    6.4690247   15.515171     3.6814928  -13.453046
    0.18186876   2.2442968  -14.809424     5.325477    -7.890806  ]
 [ -2.9269419    7.9300265   -0.476503    -2.1148179   -0.8158995
   -2.9106917   -1.7935932    0.9689238    1.1893932   -3.6501212 ]
 [ 13.68168     -2.225509     0.94028354  -3.5900285   -6.530461
   -2.2677307    1.5273005    0.9396599   -5.8294725    1.0232549 ]
 [ -0.950849    -6.3635397   -1.1536951   -3.1187148    9.486958
   -3.439334     0.48145694   1.984897    -1.1205695    3.5268595 ]
 [ -2.9552372   11.175559    -1.1968873   -3.2818384   -0.72526705
   -5.876082    -4.5625734    4.114962     0.17562361  -2.4381607 ]
 [ -4.21871     -2.621788    -1.6074116   -4.369654     9.335869
   -0.56922895  -2.30011      1.771221     3.7617486    1.6149575 ]
 [ -8.071021    -0.59627193   0.83799714   4.52211      1.2242831

In [31]:
print("예측 클래스:", y_pred)
print("진짜 클래스:", y_test[:20])

예측 클래스: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
진짜 클래스: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
